In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn 
import torch.optim as optim

import copy
import tqdm

import util
import os 

In [44]:
cwd = os.getcwd()
print(cwd)
model_path = '../model/torch_logistic_regression'

/home/linhnm/msc_code/big_data_mining/eabdt/python


In [45]:
device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda:0'
  print('ok')

ok


In [46]:
dataset = 'mix'

training_batch_from = 0
training_batch_to = 2000
val_batch_from = 3000
val_batch_to = 3600

testing_batch_from = 0
testing_batch_to = 400

In [47]:
epochs = 10000
trigger = 1000
early_stopping_round = 20

input_dim = 384
output_dim = 1
learning_rate = 0.01

In [48]:
class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(input_dim, output_dim)
  def forward(self, x):
    outputs = torch.sigmoid(self.linear(x))
    return outputs

In [49]:
model = LogisticRegression(input_dim, output_dim)
criterion = nn.BCELoss()

In [50]:
model.load_state_dict(torch.load(model_path))
model.eval()

LogisticRegression(
  (linear): Linear(in_features=384, out_features=1, bias=True)
)

In [51]:
model.to(device)

LogisticRegression(
  (linear): Linear(in_features=384, out_features=1, bias=True)
)

In [52]:
X_test = util.load_vector(os.path.join(cwd, '../data/vectorize/mix/testing_set'), testing_batch_from, testing_batch_to)
y_test = util.load_label(os.path.join(cwd, '../data/raw/mix/testing_set'), testing_batch_from, testing_batch_to)

X_gpu_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_gpu_test = torch.flatten(torch.tensor(y_test).type(torch.float32)).to(device)

In [53]:
X_gpu_test.cuda()

tensor([[-0.0297, -0.0425,  0.0579,  ..., -0.0358, -0.0148, -0.1152],
        [-0.0629,  0.0458,  0.0021,  ..., -0.0511,  0.0432, -0.0235],
        [-0.0997,  0.0008, -0.0115,  ..., -0.0179, -0.0186,  0.0597],
        ...,
        [-0.0708, -0.0321, -0.0142,  ...,  0.0142,  0.0859,  0.0578],
        [-0.0387, -0.0754,  0.0961,  ...,  0.0431, -0.0230,  0.0405],
        [-0.0274, -0.0936, -0.0093,  ...,  0.0515, -0.0042, -0.0122]],
       device='cuda:0')

In [54]:
correct_test = 0
total_test = 0
outputs_test = torch.squeeze(model(X_gpu_test))
loss_test = criterion(outputs_test, y_gpu_test)

total_test += y_gpu_test.size(0)
correct_test += torch.eq(outputs_test.round(), y_gpu_test).sum()
accuracy_test = 100 * correct_test/total_test

In [56]:
print(accuracy_test.item())

86.36325073242188
